In [13]:
import pandas as pd

file_path = 'mixalldata_clean.csv'
df = pd.read_csv(file_path, nrows=1)

print("Column names in the dataset:")
print(df.columns)

Column names in the dataset:
Index(['type', 'sendTime', 'sender', 'senderPseudo', 'messageID', 'class',
       'posx', 'posy', 'posz', 'posx_n', 'posy_n', 'posz_n', 'spdx', 'spdy',
       'spdz', 'spdx_n', 'spdy_n', 'spdz_n', 'aclx', 'acly', 'aclz', 'aclx_n',
       'acly_n', 'aclz_n', 'hedx', 'hedy', 'hedz', 'hedx_n', 'hedy_n',
       'hedz_n'],
      dtype='object')


In [12]:
import pandas as pd

# Load the dataset
file_path = 'mixalldata_clean.csv'
df = pd.read_csv(file_path)

# Specify numeric columns
numeric_columns = [
    'sendTime', 'sender', 'senderPseudo', 'posx', 'posy', 'posz', 'posx_n', 'posy_n', 'posz_n',
    'spdx', 'spdy', 'spdz', 'spdx_n', 'spdy_n', 'spdz_n',
    'aclx', 'acly', 'aclz', 'aclx_n', 'acly_n', 'aclz_n',
    'hedx', 'hedy', 'hedz', 'hedx_n', 'hedy_n', 'hedz_n', 'class'
]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Sort by 'sender' and 'sendTime' for time-sequential ordering within each sender
df = df.sort_values(by=['senderPseudo', 'sendTime']).reset_index(drop=True)

# Separate filtered records into misbehavior and normal classes
misbehavior_records = df[df['class'] != 0]
normal_records = df[df['class'] == 0]

# Sample 2000 records per misbehavior class, ensuring each sender has at least 5 occurrences
misbehavior_sample = (
    misbehavior_records.groupby('class')
    .apply(lambda x: x.head(2500))
    .reset_index(drop=True)
)

# Calculate the remaining sample size needed from normal records to reach 100,000 total
normal_sample_size = 125000 - len(misbehavior_sample)

# Sample normal records while ensuring each sender occurs at least 5 times
normal_sample = (
    normal_records.groupby('sender')
    .apply(lambda x: x.head(5))  # Take at least 5 records per sender
    .reset_index(drop=True)
)

# If additional normal records are needed, sample from remaining normal records
additional_normal_records_needed = normal_sample_size - len(normal_sample)
if additional_normal_records_needed > 0:
    remaining_normal_sample = (
        normal_records[~normal_records.index.isin(normal_sample.index)]
        .sample(n=additional_normal_records_needed, random_state=42)
    )
    normal_sample = pd.concat([normal_sample, remaining_normal_sample])

# Combine misbehavior and normal samples, and sort by 'sendTime' for a sequential dataset
final_df = pd.concat([normal_sample, misbehavior_sample]).sort_values(by='sendTime').reset_index(drop=True)

# Display final class distribution to verify balance and sender occurrence condition
print("Class distribution in final dataset:")
print(final_df['class'].value_counts())

# Verify each sender appears at least 5 times in the final dataset
sender_occurrences = final_df['sender'].value_counts()
print("All senders have at least 5 occurrences:", (sender_occurrences >= 5).all())

Class distribution in final dataset:
0     85989
18     2500
1      2500
2      2500
3      2500
4      2500
5      2500
6      2500
7      2500
8      2500
19     2500
10     2500
11     2500
12     2500
13     2500
14     2500
15     2500
16     2500
17     2500
9      2500
Name: class, dtype: int64
All senders have at least 5 occurrences: False


In [10]:
# Find all distinct (unique) values in the 'OBU_device_ID' column
distinct_senders = final_df['sender'].unique()

print("Distinct senders:")
print(distinct_senders)
print("Total senders: " + str(len(distinct_senders)))

Distinct senders:
[     9     15     21 ... 147969 147963 147981]
Total senders: 18628


In [11]:
# Display the first 10 records with sender and sendTime
pd.set_option('display.float_format', '{:.0f}'.format)
print(df[['sender', 'sendTime']].head(10))

   sender  sendTime
0    1491     16281
1    1491     16281
2    1491     16282
3    1491     16282
4    1491     16282
5    1491     16282
6    1491     16282
7    1491     16283
8    1491     16283
9    1491     16283


In [6]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']

X = df[features].values
Y = df['class'].astype(int).values

scaler = StandardScaler()
X = scaler.fit_transform(X)

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

num_classes = len(np.unique(Y))
Y = to_categorical(Y, num_classes=num_classes)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=2, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Y_pred = model.predict(X_test)

Y_pred_binary = Y_pred.argmax(axis=1)
Y_test_labels = Y_test.argmax(axis=1)

accuracy = accuracy_score(Y_test_labels, Y_pred_binary)
precision = precision_score(Y_test_labels, Y_pred_binary, average='weighted')
recall = recall_score(Y_test_labels, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test_labels, Y_pred_binary, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print(f"CNN Accuracy: {accuracy:.4f}")
print(f"CNN Precision: {precision:.4f}")
print(f"CNN Recall: {recall:.4f}")
print(f"CNN F1-score: {f1:.4f}")
print(f"\nTraining time: {elapsed_time:.6f} seconds")

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 202s 3ms/step - accuracy: 0.6951 - loss: 1.2286 - val_accuracy: 0.7220 - val_loss: 1.0981
Epoch 2/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 207s 3ms/step - accuracy: 0.7263 - loss: 1.0749 - val_accuracy: 0.7286 - val_loss: 1.0611
Epoch 3/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 216s 3ms/step - accuracy: 0.7309 - loss: 1.0504 - val_accuracy: 0.7309 - val_loss: 1.0493
Epoch 4/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 212s 3ms/step - accuracy: 0.7329 - loss: 1.0371 - val_accuracy: 0.7325 - val_loss: 1.0400
Epoch 5/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 222s 3ms/step - accuracy: 0.7345 - loss: 1.0295 - val_accuracy: 0.7334 - val_loss: 1.0359
Epoch 6/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 224s 4ms/step - accuracy: 0.7354 - loss: 1.0252 - val_accuracy: 0.7327 - val_loss: 1.0328
Epoch 7/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 223s 3ms/step - accuracy: 0.7360 - loss: 1.0205 - val_accuracy: 0.7342 - val_loss: 1.0263
Epoch 8/10
63897/63897 ━━━━━━━━━━━━━━━━━━━━ 225s 4ms/step - ac

C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.7356
CNN Precision: 0.6688
CNN Recall: 0.7356
CNN F1-score: 0.6651

Training time: 2226.826862 seconds


In [7]:
import numpy as np
import time

start_time = time.time()
random_index = np.random.randint(0, len(X_test))

sample = X_test[random_index].reshape(1, -1, 1)
true_label = Y_test[random_index]

predicted_label = model.predict(sample)

predicted_class = predicted_label.argmax(axis=1)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Predicted label (class): {predicted_class}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Randomly selected sample:
True label: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Predicted label (class): 0

Prediction time: 0.079967 seconds


In [24]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# 500-sample testing program 2
TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1, 1)
    true_label = Y_test[random_index]

    predicted_label = model.predict(sample)

    predicted_class = predicted_label.argmax(axis=1)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_class)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_12828\3578208992.py:46: RuntimeWarning: invalid value encountered in divide
  precision = np.nan_to_num(tp / (tp + fp))
C:\Users\david\AppData\Local\Temp\ipykernel_12828\3578208992.py:48: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [3]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 2
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    range_values = max_values - min_values
    range_values[range_values == 0] = 1
    scaled = 255 * (values - min_values) / range_values
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_2x2_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_2x2_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_2x2_ea.csv'.")

Images generated and saved to 'multiple_class_2x2_ea.csv'.


In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (2, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x2_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - classification_output_accuracy: 0.3785 - index_output_accuracy: 0.8188 - loss: 3.0009 - val_classification_output_accuracy: 0.4229 - val_index_output_accuracy: 0.6696 - val_loss: 3.2027
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.4699 - index_output_accuracy: 0.9392 - loss: 1.9446 - val_classification_output_accuracy: 0.4804 - val_index_output_accuracy: 0.9479 - val_loss: 1.9162
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.4939 - index_output_accuracy: 0.9521 - loss: 1.7914 - val_classification_output_accuracy: 0.4858 - val_index_output_accuracy: 0.9554 - val_loss: 1.8612
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.5187 - index_output_accuracy: 0.9587 - loss: 1.6649 - val_classification_output_accuracy: 0.4897 - val_index_output_accuracy: 0.9534 - val_loss: 1.8393
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━

In [21]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_LABELS_CSV = 'multiple_class_2x2_ea.csv'
FAC_IMAGE_SIZE = (2, 2, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Selected Image Path: veremi_multilevel_images_2x2_ea\image_8166.npy
True Classification: 1, True Misbehavior Index: 3
Predicted Classification: 1, Predicted Misbehavior Index: 3

Elapsed time: 0.055278 seconds


In [22]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x2_ea'
FAC_LABELS_CSV = 'multiple_class_2x2_ea.csv'
FAC_IMAGE_SIZE = (2, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(2, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_16772\3968340382.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [23]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 3
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    range_values = max_values - min_values
    range_values[range_values == 0] = 1
    scaled = 255 * (values - min_values) / range_values
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x3_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x3_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x3_ea.csv'.")

Images generated and saved to 'multiple_class_3x3_ea.csv'.


In [40]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x3_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - classification_output_accuracy: 0.3854 - index_output_accuracy: 0.6284 - loss: 3.7134 - val_classification_output_accuracy: 0.4905 - val_index_output_accuracy: 0.6936 - val_loss: 2.8792
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - classification_output_accuracy: 0.6221 - index_output_accuracy: 0.9177 - loss: 1.4574 - val_classification_output_accuracy: 0.6335 - val_index_output_accuracy: 0.9276 - val_loss: 1.3861
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - classification_output_accuracy: 0.6937 - index_output_accuracy: 0.9416 - loss: 1.1415 - val_classification_output_accuracy: 0.6438 - val_index_output_accuracy: 0.9294 - val_loss: 1.3985
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - classification_output_accuracy: 0.7501 - index_output_accuracy: 0.9541 - loss: 0.9272 - val_classification_output_accuracy: 0.6528 - val_index_output_accuracy: 0.9196 - val_loss: 1.5668
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━

In [41]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_LABELS_CSV = 'multiple_class_3x3_ea.csv'
FAC_IMAGE_SIZE = (3, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Selected Image Path: veremi_multilevel_images_3x3_ea\image_5334.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.058787 seconds


In [44]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x3_ea'
FAC_LABELS_CSV = 'multiple_class_3x3_ea.csv'
FAC_IMAGE_SIZE = (3, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(3, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_16772\3988713577.py:92: RuntimeWarning: invalid value encountered in divide
  recall = np.nan_to_num(tp / (tp + fn))
C:\Users\david\AppData\Local\Temp\ipykernel_16772\3988713577.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [85]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 4
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_4x4_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(2500):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_4x4_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_4x4_ea.csv'.")

Images generated and saved to 'multiple_class_4x4_ea.csv'.


In [93]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (4, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - classification_output_accuracy: 0.3977 - index_output_accuracy: 0.4922 - loss: 4.1181 - val_classification_output_accuracy: 0.3128 - val_index_output_accuracy: 0.3128 - val_loss: 7.0467
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - classification_output_accuracy: 0.6329 - index_output_accuracy: 0.8778 - loss: 1.5086 - val_classification_output_accuracy: 0.4146 - val_index_output_accuracy: 0.5341 - val_loss: 3.5682
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - classification_output_accuracy: 0.7210 - index_output_accuracy: 0.9352 - loss: 1.0734 - val_classification_output_accuracy: 0.5561 - val_index_output_accuracy: 0.8110 - val_loss: 2.1027
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - classification_output_accuracy: 0.7927 - index_output_accuracy: 0.9494 - loss: 0.7836 - val_classification_output_accuracy: 0.5921 - val_index_output_accuracy: 0.8604 - val_loss: 1.9200
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━

In [101]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_LABELS_CSV = 'multiple_class_4x4_ea.csv'
FAC_IMAGE_SIZE = (4, 4, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Selected Image Path: veremi_multilevel_images_4x4_ea\image_1476.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.059984 seconds


In [102]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x4_ea'
FAC_LABELS_CSV = 'multiple_class_4x4_ea.csv'
FAC_IMAGE_SIZE = (4, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(4, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_208\1508592245.py:92: RuntimeWarning: invalid value encountered in divide
  recall = np.nan_to_num(tp / (tp + fn))
C:\Users\david\AppData\Local\Temp\ipykernel_208\1508592245.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [28]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_5x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(2500):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_5x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_5x5_ea.csv'.")

Images generated and saved to 'multiple_class_5x5_ea.csv'.


In [31]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - classification_output_accuracy: 0.4766 - index_output_accuracy: 0.4701 - loss: 3.7854 - val_classification_output_accuracy: 0.3799 - val_index_output_accuracy: 0.3183 - val_loss: 5.1627
Epoch 2/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - classification_output_accuracy: 0.8941 - index_output_accuracy: 0.9732 - loss: 0.5514 - val_classification_output_accuracy: 0.4866 - val_index_output_accuracy: 0.4177 - val_loss: 3.7573
Epoch 3/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - classification_output_accuracy: 0.9711 - index_output_accuracy: 0.9962 - loss: 0.1894 - val_classification_output_accuracy: 0.8122 - val_index_output_accuracy: 0.8811 - val_loss: 1.0643
Epoch 4/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - classification_output_accuracy: 0.9962 - index_output_accuracy: 0.9985 - loss: 0.0809 - val_classification_output_accuracy: 0.8780 - val_index_output_accuracy: 0.9524 - val_loss: 0.5526
Epoch 5/10
205/205 ━━━━━━━━━━━━━━━━━━━━ 

In [71]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_LABELS_CSV = 'multiple_class_5x5_ea.csv'
FAC_IMAGE_SIZE = (5, 5, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Selected Image Path: veremi_multilevel_images_5x5_ea\image_2224.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.059327 seconds


In [60]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x5_ea'
FAC_LABELS_CSV = 'multiple_class_5x5_ea.csv'
FAC_IMAGE_SIZE = (5, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

In [13]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x2_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        chunk = group.sample(n=2, random_state=image_counter)

        image_data = records_to_image(chunk)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            normal_chunk = group.sample(n=1, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 3)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class, 
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x2_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x2_ea.csv'.")

Classification results saved to 'multiple_class_1x2_ea.csv'.


In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x2_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(3, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - classification_output_accuracy: 0.4725 - index_output_accuracy: 0.9882 - loss: 1.8128 - val_classification_output_accuracy: 0.4662 - val_index_output_accuracy: 1.0000 - val_loss: 1.7787
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - classification_output_accuracy: 0.5117 - index_output_accuracy: 0.9997 - loss: 1.5684 - val_classification_output_accuracy: 0.4936 - val_index_output_accuracy: 1.0000 - val_loss: 1.6574
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.5281 - index_output_accuracy: 1.0000 - loss: 1.5095 - val_classification_output_accuracy: 0.4812 - val_index_output_accuracy: 1.0000 - val_loss: 1.8510
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.5305 - index_output_accuracy: 0.9999 - loss: 1.4900 - val_classification_output_accuracy: 0.4925 - val_index_output_accuracy: 1.0000 - val_loss: 1.6976
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━

In [21]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_LABELS_CSV = 'multiple_class_1x2_ea.csv'
FAC_IMAGE_SIZE = (1, 2)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 2)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
True Classification: 15, True Misbehavior Index: 1
Predicted Classification: 15, Predicted Misbehavior Index: 1

Elapsed time: 0.056844 seconds


In [22]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x2_ea'
FAC_LABELS_CSV = 'multiple_class_1x2_ea.csv'
FAC_IMAGE_SIZE = (1, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━

In [219]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x3_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 3:
            group = group.sample(n=3, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=3, random_state=image_counter)

        image_data = records_to_image(chunk)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            normal_chunk = group.sample(n=2, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 4)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class, 
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x3_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x3_ea.csv'.")

Classification results saved to 'multiple_class_1x3_ea.csv'.


In [25]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x3_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(4, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - classification_output_accuracy: 0.4670 - index_output_accuracy: 0.9897 - loss: 1.8382 - val_classification_output_accuracy: 0.4724 - val_index_output_accuracy: 0.9997 - val_loss: 1.8913
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - classification_output_accuracy: 0.5152 - index_output_accuracy: 0.9998 - loss: 1.5751 - val_classification_output_accuracy: 0.4856 - val_index_output_accuracy: 0.9997 - val_loss: 1.7096
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - classification_output_accuracy: 0.5266 - index_output_accuracy: 0.9998 - loss: 1.5085 - val_classification_output_accuracy: 0.4959 - val_index_output_accuracy: 0.9997 - val_loss: 1.6903
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - classification_output_accuracy: 0.5388 - index_output_accuracy: 0.9997 - loss: 1.4598 - val_classification_output_accuracy: 0.5178 - val_index_output_accuracy: 0.9997 - val_loss: 1.5422
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━

In [32]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_LABELS_CSV = 'multiple_class_1x3_ea.csv'
FAC_IMAGE_SIZE = (1, 3)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 3)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
True Classification: 18, True Misbehavior Index: 3
Predicted Classification: 18, Predicted Misbehavior Index: 3

Elapsed time: 0.059296 seconds


In [34]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x3_ea'
FAC_LABELS_CSV = 'multiple_class_1x3_ea.csv'
FAC_IMAGE_SIZE = (1, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━

In [209]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x4_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(8000):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 4:
            group = group.sample(n=4, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=4, random_state=image_counter)

        image_data = records_to_image(group)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({
            'npy_file_name': npy_file_name, 
            'classification': 0, 
            'misbehavior_index': misbehavior_index
        })

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(600):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            if len(group) < 3:
                normal_chunk = group.sample(n=3, replace=True, random_state=image_counter)
            else:
                normal_chunk = group.sample(n=3, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)
            
            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 5)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)
           
            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class,  # Misbehavior class value assigned
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x4_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x4_ea.csv'.")

Classification results saved to 'multiple_class_1x4_ea.csv'.


In [35]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_1x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(5, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - classification_output_accuracy: 0.4574 - index_output_accuracy: 0.9888 - loss: 1.8955 - val_classification_output_accuracy: 0.4722 - val_index_output_accuracy: 1.0000 - val_loss: 1.8083
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.5065 - index_output_accuracy: 0.9998 - loss: 1.5957 - val_classification_output_accuracy: 0.4920 - val_index_output_accuracy: 1.0000 - val_loss: 1.6385
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.5302 - index_output_accuracy: 0.9997 - loss: 1.4971 - val_classification_output_accuracy: 0.5116 - val_index_output_accuracy: 0.9990 - val_loss: 1.6647
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - classification_output_accuracy: 0.5550 - index_output_accuracy: 0.9997 - loss: 1.4180 - val_classification_output_accuracy: 0.5268 - val_index_output_accuracy: 1.0000 - val_loss: 1.5683
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━

In [42]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_LABELS_CSV = 'multiple_class_1x4_ea.csv'
FAC_IMAGE_SIZE = (1, 4)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 4)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
True Classification: 6, True Misbehavior Index: 4
Predicted Classification: 6, Predicted Misbehavior Index: 4

Elapsed time: 0.061949 seconds


In [43]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x4_ea'
FAC_LABELS_CSV = 'multiple_class_1x4_ea.csv'
FAC_IMAGE_SIZE = (1, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━

C:\Users\david\AppData\Local\Temp\ipykernel_12424\1744340218.py:93: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [17]:
import numpy as np
import pandas as pd
import os

FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 1
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

min_max_values = {
    col: (final_df[col].min(), final_df[col].max()) for col in final_numeric_columns if col != 'class'
}

def normalize_and_scale(value, min_value, max_value):
    if pd.isna(value) or not np.isfinite(value):
        return FAC_SPECIAL_VALUE
    if max_value == min_value:
        return 0 if value == min_value else FAC_SPECIAL_VALUE
    scaled_value = 255 * (value - min_value) / (max_value - min_value)
    return int(np.clip(scaled_value, 0, 255))

def records_to_image(group, misbehavior_index=None):
    image_data = np.zeros((FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS), dtype=np.uint8)

    group = group.sort_values('sendTime')
    
    for i, (_, row) in enumerate(group.iterrows()):
        row_pos = 0
        col_pos = i % FAC_IMAGE_WIDTH
        
        for j, col in enumerate(final_numeric_columns[:-1]):
            min_value, max_value = min_max_values[col]
            value = normalize_and_scale(row[col], min_value, max_value)
            image_data[row_pos, col_pos, j] = value

    if misbehavior_index is not None:
        misbehavior_row = 0
        misbehavior_col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        image_data[misbehavior_row, misbehavior_col, 0] = 255
        image_data[misbehavior_row, misbehavior_col, 1] = 0

    return image_data

output_dir = 'veremi_multilevel_images_1x5_ea'
os.makedirs(output_dir, exist_ok=True)

classification_results = []
image_counter = 1

normal_messages = final_df[final_df['class'] == 0]
attack_messages = final_df[final_df['class'] != 0]

for _ in range(2500):
    for obu_id, group in normal_messages.groupby('senderPseudo'):
        if len(group) < 5:
            group = group.sample(n=5, replace=True, random_state=image_counter)
        else:
            group = group.sample(n=5, random_state=image_counter)

        # Pass 'group' instead of 'chunk'
        image_data = records_to_image(group)
        misbehavior_index = 0

        npy_file_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_file_name, image_data)

        classification_results.append({'npy_file_name': npy_file_name, 'classification': 0, 'misbehavior_index': misbehavior_index})

        image_counter += 1
        break

for misbehavior_class in range(1, 20):
    for _ in range(400):
        for obu_id, group in normal_messages.groupby('senderPseudo'):
            if len(group) < 4:
                normal_chunk = group.sample(n=4, replace=True, random_state=image_counter)
            else:
                normal_chunk = group.sample(n=4, random_state=image_counter)
            attack_chunk = attack_messages[attack_messages['class'] == misbehavior_class].sample(n=1, random_state=image_counter)

            combined_chunk = pd.concat([normal_chunk, attack_chunk])

            misbehavior_index = np.random.randint(1, 6)
            image_data = records_to_image(combined_chunk, misbehavior_index)

            npy_file_name = f'{output_dir}/image_{image_counter}.npy'
            np.save(npy_file_name, image_data)

            classification_results.append({
                'npy_file_name': npy_file_name, 
                'classification': misbehavior_class,
                'misbehavior_index': misbehavior_index
            })

            image_counter += 1
            break

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_1x5_ea.csv', index=False)

print("Classification results saved to 'multiple_class_1x5_ea.csv'.")

Classification results saved to 'multiple_class_1x5_ea.csv'.


In [33]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (1, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 20

fac_labels_df = pd.read_csv('multiple_class_1x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)

input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(6, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping, lr_scheduler])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro')
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro')
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro')

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - classification_output_accuracy: 0.3096 - index_output_accuracy: 0.9785 - loss: 2.6205 - val_classification_output_accuracy: 0.2911 - val_index_output_accuracy: 0.2530 - val_loss: 4.8329 - learning_rate: 0.0010
Epoch 2/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - classification_output_accuracy: 0.3407 - index_output_accuracy: 0.9999 - loss: 2.1742 - val_classification_output_accuracy: 0.3332 - val_index_output_accuracy: 0.9832 - val_loss: 2.3984 - learning_rate: 0.0010
Epoch 3/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - classification_output_accuracy: 0.3918 - index_output_accuracy: 1.0000 - loss: 1.9795 - val_classification_output_accuracy: 0.3901 - val_index_output_accuracy: 0.9837 - val_loss: 2.0556 - learning_rate: 0.0010
Epoch 4/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - classification_output_accuracy: 0.4023 - index_output_accuracy: 0.9996 - loss: 1.9342 - val_classification_output_accuracy: 0.3554 - val_index_output_

In [38]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_LABELS_CSV = 'multiple_class_1x5_ea.csv'
FAC_IMAGE_SIZE = (1, 5)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

def preprocess_image(file_path, target_size=(1, 5)):
    """Load and preprocess a .npy image file for prediction."""
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

image_array = preprocess_image(selected_image_path)
image_array = np.expand_dims(image_array, axis=0)

predictions = model.predict(image_array)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

if not os.path.exists(selected_image_path):
    raise FileNotFoundError(f"File {selected_image_path} not found.")

start_time = time.time()
try:
    image_array = preprocess_image(selected_image_path)
    image_array = np.expand_dims(image_array, axis=0)
    predictions = model.predict(image_array)
    predicted_classification = np.argmax(predictions[0])
    predicted_index = np.argmax(predictions[1])
    
    print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
    print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
except Exception as e:
    print(f"Error: {e}")
end_time = time.time()

print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
True Classification: 9, True Misbehavior Index: 2
Predicted Classification: 9, Predicted Misbehavior Index: 2

Elapsed time: 0.064681 seconds


In [39]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_1x5_ea'
FAC_LABELS_CSV = 'multiple_class_1x5_ea.csv'
FAC_IMAGE_SIZE = (1, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(1, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.divide(tp, (tp + fp), out=np.zeros_like(tp, dtype=float), where=(tp + fp) != 0)
    recall = np.divide(tp, (tp + fn), out=np.zeros_like(tp, dtype=float), where=(tp + fn) != 0)
    f1 = np.divide(2 * (precision * recall), (precision + recall), 
                   out=np.zeros_like(precision, dtype=float), where=(precision + recall) != 0)

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━

In [234]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 2
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_2x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(8000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_2x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_2x5_ea.csv'.")

Images generated and saved to 'multiple_class_2x5_ea.csv'.


In [239]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 2, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_2x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - classification_output_accuracy: 0.4533 - index_output_accuracy: 0.7068 - loss: 3.0816 - val_classification_output_accuracy: 0.4948 - val_index_output_accuracy: 0.6673 - val_loss: 3.0492
Epoch 2/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - classification_output_accuracy: 0.6133 - index_output_accuracy: 0.9260 - loss: 1.4394 - val_classification_output_accuracy: 0.6028 - val_index_output_accuracy: 0.9332 - val_loss: 1.5450
Epoch 3/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.6555 - index_output_accuracy: 0.9505 - loss: 1.2169 - val_classification_output_accuracy: 0.6353 - val_index_output_accuracy: 0.9528 - val_loss: 1.3573
Epoch 4/10
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.7013 - index_output_accuracy: 0.9600 - loss: 1.0369 - val_classification_output_accuracy: 0.6696 - val_index_output_accuracy: 0.9549 - val_loss: 1.2018
Epoch 5/10
485/485 ━━━━━━━━━━━━━━━━━

In [240]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_LABELS_CSV = 'multiple_class_2x5_ea.csv'
FAC_IMAGE_SIZE = (5, 2, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Selected Image Path: veremi_multilevel_images_2x5_ea\image_18749.npy
True Classification: 18, True Misbehavior Index: 2
Predicted Classification: 18, Predicted Misbehavior Index: 2

Elapsed time: 0.068709 seconds


In [242]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_2x5_ea'
FAC_LABELS_CSV = 'multiple_class_2x5_ea.csv'
FAC_IMAGE_SIZE = (5, 2)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 2)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [243]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 3
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(600):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x5_ea.csv'.")

Images generated and saved to 'multiple_class_3x5_ea.csv'.


In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 3, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - classification_output_accuracy: 0.3693 - index_output_accuracy: 0.5554 - loss: 3.7919 - val_classification_output_accuracy: 0.2674 - val_index_output_accuracy: 0.4250 - val_loss: 4.8462
Epoch 2/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - classification_output_accuracy: 0.6065 - index_output_accuracy: 0.8878 - loss: 1.5458 - val_classification_output_accuracy: 0.5924 - val_index_output_accuracy: 0.8743 - val_loss: 1.6408
Epoch 3/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - classification_output_accuracy: 0.6867 - index_output_accuracy: 0.9347 - loss: 1.1301 - val_classification_output_accuracy: 0.6076 - val_index_output_accuracy: 0.9010 - val_loss: 1.6699
Epoch 4/10
360/360 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - classification_output_accuracy: 0.7492 - index_output_accuracy: 0.9516 - loss: 0.9072 - val_classification_output_accuracy: 0.6281 - val_index_output_accuracy: 0.9149 - val_loss: 1.5159
Epoch 5/10
360/360 ━━━━━━━━━━━━━━━━━

In [8]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_LABELS_CSV = 'multiple_class_3x5_ea.csv'
FAC_IMAGE_SIZE = (5, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Selected Image Path: veremi_multilevel_images_3x5_ea\image_4353.npy
True Classification: 3, True Misbehavior Index: 15
Predicted Classification: 10, Predicted Misbehavior Index: 15

Elapsed time: 0.063639 seconds


In [9]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x5_ea'
FAC_LABELS_CSV = 'multiple_class_3x5_ea.csv'
FAC_IMAGE_SIZE = (5, 3)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 3)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

In [15]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 5
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_4x5_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(500):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_4x5_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_4x5_ea.csv'.")

Images generated and saved to 'multiple_class_4x5_ea.csv'.


In [17]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (5, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_4x5_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - classification_output_accuracy: 0.4615 - index_output_accuracy: 0.5070 - loss: 3.7033 - val_classification_output_accuracy: 0.3020 - val_index_output_accuracy: 0.3208 - val_loss: 5.5531
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - classification_output_accuracy: 0.6930 - index_output_accuracy: 0.8843 - loss: 1.2887 - val_classification_output_accuracy: 0.6532 - val_index_output_accuracy: 0.8576 - val_loss: 1.5790
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - classification_output_accuracy: 0.7805 - index_output_accuracy: 0.9326 - loss: 0.8801 - val_classification_output_accuracy: 0.6592 - val_index_output_accuracy: 0.8836 - val_loss: 1.4685
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - classification_output_accuracy: 0.8373 - index_output_accuracy: 0.9560 - loss: 0.6123 - val_classification_output_accuracy: 0.7096 - val_index_output_accuracy: 0.9004 - val_loss: 1.3159
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━

In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_LABELS_CSV = 'multiple_class_4x5_ea.csv'
FAC_IMAGE_SIZE = (5, 4, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Selected Image Path: veremi_multilevel_images_4x5_ea\image_3071.npy
True Classification: 1, True Misbehavior Index: 16
Predicted Classification: 10, Predicted Misbehavior Index: 16

Elapsed time: 0.073412 seconds


In [19]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_4x5_ea'
FAC_LABELS_CSV = 'multiple_class_4x5_ea.csv'
FAC_IMAGE_SIZE = (5, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(5, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━

In [7]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 4
FAC_IMAGE_HEIGHT = 3
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_3x4_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(500):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_3x4_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_3x4_ea.csv'.")

Images generated and saved to 'multiple_class_3x4_ea.csv'.


In [30]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (3, 4, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_3x4_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(26, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - classification_output_accuracy: 0.3393 - index_output_accuracy: 0.5686 - loss: 3.9106 - val_classification_output_accuracy: 0.2568 - val_index_output_accuracy: 0.2892 - val_loss: 6.5018
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - classification_output_accuracy: 0.5512 - index_output_accuracy: 0.8925 - loss: 1.7534 - val_classification_output_accuracy: 0.5420 - val_index_output_accuracy: 0.8608 - val_loss: 1.9360
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - classification_output_accuracy: 0.6390 - index_output_accuracy: 0.9206 - loss: 1.3539 - val_classification_output_accuracy: 0.5340 - val_index_output_accuracy: 0.8836 - val_loss: 1.9753
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - classification_output_accuracy: 0.6928 - index_output_accuracy: 0.9526 - loss: 1.0723 - val_classification_output_accuracy: 0.5352 - val_index_output_accuracy: 0.9004 - val_loss: 2.0362
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━

In [37]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_LABELS_CSV = 'multiple_class_3x4_ea.csv'
FAC_IMAGE_SIZE = (4, 3, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Selected Image Path: veremi_multilevel_images_3x4_ea\image_6927.npy
True Classification: 8, True Misbehavior Index: 7
Predicted Classification: 8, Predicted Misbehavior Index: 7

Elapsed time: 0.062025 seconds


In [40]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_3x4_ea'
FAC_LABELS_CSV = 'multiple_class_3x4_ea.csv'
FAC_IMAGE_SIZE = (3, 4)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(3, 4)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

In [41]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 6
FAC_IMAGE_HEIGHT = 6
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_6x6_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_6x6_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_6x6_ea.csv'.")

Images generated and saved to 'multiple_class_6x6_ea.csv'.


In [45]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 6, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_6x6_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(37, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - classification_output_accuracy: 0.5786 - index_output_accuracy: 0.3996 - loss: 3.8764 - val_classification_output_accuracy: 0.3592 - val_index_output_accuracy: 0.3506 - val_loss: 8.6585
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - classification_output_accuracy: 0.8042 - index_output_accuracy: 0.8534 - loss: 1.0686 - val_classification_output_accuracy: 0.5718 - val_index_output_accuracy: 0.6213 - val_loss: 2.7645
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - classification_output_accuracy: 0.8943 - index_output_accuracy: 0.9405 - loss: 0.5441 - val_classification_output_accuracy: 0.6310 - val_index_output_accuracy: 0.8207 - val_loss: 2.2136
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - classification_output_accuracy: 0.9392 - index_output_accuracy: 0.9744 - loss: 0.3034 - val_classification_output_accuracy: 0.8029 - val_index_output_accuracy: 0.8517 - val_loss: 1.1857
Epoch 5/10
218/218 ━━━━━━━━━━━━━━━━━

In [50]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_LABELS_CSV = 'multiple_class_6x6_ea.csv'
FAC_IMAGE_SIZE = (6, 6, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Selected Image Path: veremi_multilevel_images_6x6_ea\image_3562.npy
True Classification: 2, True Misbehavior Index: 34
Predicted Classification: 2, Predicted Misbehavior Index: 34

Elapsed time: 0.057880 seconds


In [51]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_6x6_ea'
FAC_LABELS_CSV = 'multiple_class_6x6_ea.csv'
FAC_IMAGE_SIZE = (6, 6)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(6, 6)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━

In [52]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 5
FAC_IMAGE_HEIGHT = 6
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_5x6_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_5x6_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_5x6_ea.csv'.")

Images generated and saved to 'multiple_class_5x6_ea.csv'.


In [55]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (6, 5, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_5x6_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(31, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - classification_output_accuracy: 0.5431 - index_output_accuracy: 0.4152 - loss: 3.8593 - val_classification_output_accuracy: 0.3713 - val_index_output_accuracy: 0.3506 - val_loss: 5.7640
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - classification_output_accuracy: 0.7888 - index_output_accuracy: 0.8659 - loss: 1.0857 - val_classification_output_accuracy: 0.5138 - val_index_output_accuracy: 0.7540 - val_loss: 2.9089
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - classification_output_accuracy: 0.8765 - index_output_accuracy: 0.9442 - loss: 0.5885 - val_classification_output_accuracy: 0.7609 - val_index_output_accuracy: 0.8408 - val_loss: 1.3326
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - classification_output_accuracy: 0.9156 - index_output_accuracy: 0.9683 - loss: 0.3816 - val_classification_output_accuracy: 0.7310 - val_index_output_accuracy: 0.8328 - val_loss: 1.6503
Epoch 5/10
218/218 ━━━━━━━━━━━━━━━━━

In [64]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_LABELS_CSV = 'multiple_class_5x6_ea.csv'
FAC_IMAGE_SIZE = (6, 5, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Selected Image Path: veremi_multilevel_images_5x6_ea\image_328.npy
True Classification: 0, True Misbehavior Index: 0
Predicted Classification: 0, Predicted Misbehavior Index: 0

Elapsed time: 0.069336 seconds


In [66]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_5x6_ea'
FAC_LABELS_CSV = 'multiple_class_5x6_ea.csv'
FAC_IMAGE_SIZE = (6, 5)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(6, 5)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

In [67]:
import numpy as np
import pandas as pd
import os

# Constants
FAC_IMAGE_WIDTH = 7
FAC_IMAGE_HEIGHT = 7
FAC_NUM_CHANNELS = 12
FAC_SPECIAL_VALUE = 255
NUM_CLASSES = 20

def normalize_and_scale(values, min_values, max_values):
    scaled = 255 * (values - min_values) / (max_values - min_values)
    scaled[np.isnan(values) | ~np.isfinite(values)] = FAC_SPECIAL_VALUE
    scaled[values == max_values] = 0
    return np.clip(scaled, 0, 255).astype(np.uint8)

def generate_image(group, misbehavior_index=None):
    min_values = np.array([group[col].min() for col in final_numeric_columns])
    max_values = np.array([group[col].max() for col in final_numeric_columns])

    data = group[final_numeric_columns].to_numpy()
    normalized_data = normalize_and_scale(data, min_values, max_values)

    reshaped_data = normalized_data.reshape(FAC_IMAGE_HEIGHT, FAC_IMAGE_WIDTH, FAC_NUM_CHANNELS)

    if misbehavior_index:
        row = (misbehavior_index - 1) // FAC_IMAGE_WIDTH
        col = (misbehavior_index - 1) % FAC_IMAGE_WIDTH
        reshaped_data[row, col, :3] = [255, 0, 0]
        
    return reshaped_data

final_numeric_columns = [
    'sendTime', 'senderPseudo', 'posx', 'posy', 'posx_n', 'posy_n',
    'spdx_n', 'spdy_n', 'aclx_n', 'acly_n', 'hedx_n', 'hedy_n'
]

output_dir = 'veremi_multilevel_images_7x7_ea'
os.makedirs(output_dir, exist_ok=True)

normal_messages = final_df[final_df['class'] == 0]

classification_results = []
image_counter = 1

for _ in range(3000):
    sampled_group = normal_messages.sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
    image_data = generate_image(sampled_group)
    npy_name = f'{output_dir}/image_{image_counter}.npy'
    np.save(npy_name, image_data)
    classification_results.append({'file_name': npy_name, 'classification': 0, 'misbehavior_index': 0})
    image_counter += 1

for misbehavior_class in range(1, 20):
    for _ in range(300):
        attack_group = final_df[final_df['class'] == misbehavior_class].sample(n=FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT, random_state=image_counter)
        misbehavior_index = np.random.randint(1, FAC_IMAGE_WIDTH * FAC_IMAGE_HEIGHT + 1)
        image_data = generate_image(attack_group, misbehavior_index)
        npy_name = f'{output_dir}/image_{image_counter}.npy'
        np.save(npy_name, image_data)
        classification_results.append({'file_name': npy_name, 'classification': misbehavior_class, 'misbehavior_index': misbehavior_index})
        image_counter += 1

classification_df = pd.DataFrame(classification_results)
classification_df.to_csv('multiple_class_7x7_ea.csv', index=False)

print("Images generated and saved to 'multiple_class_7x7_ea.csv'.")

Images generated and saved to 'multiple_class_7x7_ea.csv'.


In [69]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import time

FAC_IMAGE_SIZE = (7, 7, 12)
FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_BATCH_SIZE = 32
FAC_EPOCHS = 10

fac_labels_df = pd.read_csv('multiple_class_7x7_ea.csv')

fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

fac_labels = fac_labels_df[['classification', 'misbehavior_index']].values

valid_indices = (fac_labels[:, 0] != -1) & (fac_labels[:, 1] != -1)
fac_labels = fac_labels[valid_indices]
fac_num_images = len(fac_labels)

def load_images(fac_image_dir, fac_num_images):
    X = []
    Y_classification = []
    Y_index = []
    
    for idx in range(fac_num_images):
        image_path = os.path.join(fac_image_dir, f'image_{idx + 1}.npy')
        if os.path.exists(image_path):
            image_array = np.load(image_path)
            image_array = image_array / 255.0

            X.append(image_array)
            Y_classification.append(int(fac_labels[idx][0]))
            Y_index.append(int(fac_labels[idx][1]))
        else:
            print(f"Warning: {image_path} not found.")
    
    X = np.array(X, dtype=np.float32)
    Y_classification = np.array(Y_classification, dtype=int)
    Y_index = np.array(Y_index, dtype=int)

    return X, Y_classification, Y_index

X, Y_classification, Y_index = load_images(FAC_IMAGE_DIR, fac_num_images)

X_train, X_test, Y_train_classification, Y_test_classification, Y_train_index, Y_test_index = train_test_split(
    X, Y_classification, Y_index, test_size=0.2, random_state=42)
input_layer = Input(shape=(FAC_IMAGE_SIZE[0], FAC_IMAGE_SIZE[1], 12))

x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)

classification_output = Dense(20, activation='softmax', name='classification_output')(x)

index_output = Dense(50, activation='softmax', name='index_output')(x)

model = Model(inputs=input_layer, outputs=[classification_output, index_output])

model.compile(optimizer='adam', 
              loss={'classification_output': 'sparse_categorical_crossentropy', 
                    'index_output': 'sparse_categorical_crossentropy'},
              metrics={'classification_output': 'accuracy', 'index_output': 'accuracy'})

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, 
          {'classification_output': Y_train_classification, 'index_output': Y_train_index}, 
          epochs=FAC_EPOCHS, 
          batch_size=FAC_BATCH_SIZE, 
          validation_data=(X_test, {'classification_output': Y_test_classification, 'index_output': Y_test_index}),
          callbacks=[early_stopping])

predictions = model.predict(X_test)
Y_pred_classification = np.argmax(predictions[0], axis=1)
Y_pred_index = np.argmax(predictions[1], axis=1)

accuracy_classification = accuracy_score(Y_test_classification, Y_pred_classification)
precision_classification = precision_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
recall_classification = recall_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)
f1_classification = f1_score(Y_test_classification, Y_pred_classification, average='macro', zero_division=0)

accuracy_index = accuracy_score(Y_test_index, Y_pred_index)
f1_classification_manual = 2 * (precision_classification * recall_classification) / (precision_classification + recall_classification)

print("Classification Accuracy:", accuracy_classification)
print("Classification Precision:", precision_classification)
print("Classification Recall:", recall_classification)
print("Classification F1-score (sklearn):", f1_classification)
print("Classification F1-score (manual calculation):", f1_classification_manual)

print("Misbehavior Index Accuracy:", accuracy_index)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - classification_output_accuracy: 0.6459 - index_output_accuracy: 0.3533 - loss: 4.0275 - val_classification_output_accuracy: 0.3506 - val_index_output_accuracy: 0.3506 - val_loss: 16.6335
Epoch 2/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - classification_output_accuracy: 0.8482 - index_output_accuracy: 0.8010 - loss: 1.2129 - val_classification_output_accuracy: 0.4902 - val_index_output_accuracy: 0.6040 - val_loss: 3.8210
Epoch 3/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - classification_output_accuracy: 0.9103 - index_output_accuracy: 0.9297 - loss: 0.5428 - val_classification_output_accuracy: 0.7925 - val_index_output_accuracy: 0.7920 - val_loss: 1.4582
Epoch 4/10
218/218 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - classification_output_accuracy: 0.9631 - index_output_accuracy: 0.9731 - loss: 0.2371 - val_classification_output_accuracy: 0.7914 - val_index_output_accuracy: 0.8103 - val_loss: 1.5066
Epoch 5/10
218/218 ━━━━━━━━━━━━━━━

In [79]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_LABELS_CSV = 'multiple_class_7x7_ea.csv'
FAC_IMAGE_SIZE = (7, 7, 12)

def preprocess_image(file_path):
    """
    Load and preprocess an image for prediction.
    Expects .npy files containing image arrays.
    """
    image_array = np.load(file_path) / 255.0
    return np.expand_dims(image_array, axis=0)

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)
valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

random_index = np.random.choice(valid_labels.index)
selected_label = valid_labels.loc[random_index]
selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

start_time = time.time()
image_array = preprocess_image(selected_image_path)

predictions = model.predict(image_array)
predicted_classification = np.argmax(predictions[0])
predicted_index = np.argmax(predictions[1])
end_time = time.time()

print(f"Selected Image Path: {selected_image_path}")
print(f"True Classification: {selected_label['classification']}, True Misbehavior Index: {selected_label['misbehavior_index']}")
print(f"Predicted Classification: {predicted_classification}, Predicted Misbehavior Index: {predicted_index}")
print(f"\nElapsed time: {end_time - start_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Selected Image Path: veremi_multilevel_images_7x7_ea\image_3004.npy
True Classification: 1, True Misbehavior Index: 12
Predicted Classification: 1, Predicted Misbehavior Index: 12

Elapsed time: 0.072966 seconds


In [80]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import os
import time

FAC_IMAGE_DIR = 'veremi_multilevel_images_7x7_ea'
FAC_LABELS_CSV = 'multiple_class_7x7_ea.csv'
FAC_IMAGE_SIZE = (7, 7)
NUM_SAMPLES = 500
TIME_THRESHOLD = 0.1

fac_labels_df = pd.read_csv(FAC_LABELS_CSV)
fac_labels_df['classification'].fillna(-1, inplace=True)
fac_labels_df['misbehavior_index'].fillna(-1, inplace=True)

valid_indices = (fac_labels_df['classification'] != -1) & (fac_labels_df['misbehavior_index'] != -1)
valid_labels = fac_labels_df[valid_indices]

def preprocess_image(file_path, target_size=(7, 7)):
    try:
        image = np.load(file_path)
        if image.shape[:2] != target_size:
            raise ValueError(f"Unexpected image size: {image.shape[:2]}, expected {target_size}")
        image_array = image / 255.0
        if len(image_array.shape) == 2:
            image_array = np.expand_dims(image_array, axis=-1)
        return image_array
    except Exception as e:
        raise ValueError(f"Error processing file {file_path}: {e}")

total_classification_correct = 0
total_index_correct = 0
total_prediction_time = 0.0
predictions_above_threshold = 0
total_samples_processed = 0

true_classifications = []
predicted_classifications = []
true_indices = []
predicted_indices = []

for _ in range(NUM_SAMPLES):
    random_index = np.random.choice(valid_labels.index)
    selected_label = valid_labels.loc[random_index]
    selected_image_path = os.path.join(FAC_IMAGE_DIR, f'image_{random_index + 1}.npy')

    if not os.path.exists(selected_image_path):
        continue

    try:
        start_time = time.time()
        image_array = preprocess_image(selected_image_path)
        image_array = np.expand_dims(image_array, axis=0)

        predictions = model.predict(image_array)
        predicted_classification = np.argmax(predictions[0])
        predicted_index = np.argmax(predictions[1])

        true_classifications.append(int(selected_label['classification']))
        predicted_classifications.append(predicted_classification)
        true_indices.append(int(selected_label['misbehavior_index']))
        predicted_indices.append(predicted_index)

        if predicted_classification == int(selected_label['classification']):
            total_classification_correct += 1
        if predicted_index == int(selected_label['misbehavior_index']):
            total_index_correct += 1

        prediction_time = time.time() - start_time
        total_prediction_time += prediction_time

        if prediction_time >= TIME_THRESHOLD:
            predictions_above_threshold += 1

        total_samples_processed += 1

    except Exception as e:
        print(f"Error during prediction: {e}")

classification_cm = confusion_matrix(true_classifications, predicted_classifications)
index_cm = confusion_matrix(true_indices, predicted_indices)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp
    total = np.sum(cm)

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    accuracy = np.sum(tp) / total if total > 0 else 0.0

    return accuracy, precision, recall, f1

classification_accuracy, classification_precision, classification_recall, classification_f1 = calculate_metrics(classification_cm)
index_accuracy, index_precision, index_recall, index_f1 = calculate_metrics(index_cm)

print("\nClassification Metrics:")
print(f"Accuracy: {classification_accuracy:.2f}")
print(f"Average Precision: {np.mean(classification_precision):.2f}")
print(f"Average Recall: {np.mean(classification_recall):.2f}")
print(f"Average F1-Score: {np.mean(classification_f1):.2f}")

print("\nIndex Prediction Metrics:")
print(f"Accuracy: {index_accuracy:.2f}")
print(f"Average Precision: {np.mean(index_precision):.2f}")
print(f"Average Recall: {np.mean(index_recall):.2f}")
print(f"Average F1-Score: {np.mean(index_f1):.2f}")

print(f"\nAverage Prediction Time: {total_prediction_time / total_samples_processed:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━